In [1]:
import Pkg; Pkg.activate(joinpath(@__DIR__,"..")); Pkg.instantiate()
using LinearAlgebra
using ForwardDiff
using RobotDynamics
using Ipopt
using MathOptInterface
using TrajOptPlots
const MOI = MathOptInterface
using Random

  Activating environment at `~/planar_quadruped_landing/Project.toml`


In [21]:
include("quadratic_cost.jl")
include("planar_quadruped.jl")
include("sparseblocks.jl")
include("utils.jl")
include("nlp.jl")
include("moi.jl")
include("costs.jl")
include("constraints.jl")
include("ref_traj.jl")

reference_trajectory

## Reference Trajectory

## Problem Definition

In [22]:
# Dynamics model
model = PlanarQuadruped()
g, mb, lb, l1, l2 = model.g, model.mb, model.lb, model.l1, model.l2

# Discretization
tf = 0.6
dt = 0.02
N = Int(ceil(tf/dt)) + 1
times = range(0,tf, length=N)
t_trans = 0.2

n = 14
m = 4

# Initial Conditions
# currently, we assume the initial mode ID is 1
# xinit = [-0.4;0.6;-3.1415/9;        0.0;0.0; -1.0;0.2; 0.0;-0.5;0.0; 0.0;0.0; 0.0;-0.5]
# xterm = [-lb/2;sqrt(l1^2+l2^2);0.0; 0.0;0.0; -lb;0.0;  0.0;0.0;0.0;  0.0;0.0; 0.0;0.0]
xinit = zeros(n)
xinit[1] = -lb/2                  # xb
xinit[2] = sqrt(l1^2+l2^2) + 0.05 # yb
xinit[3] = 10 * pi / 180          # theta
xinit[6] = -lb                    # x2
xinit[7] = 0.05                   # y2
xinit[9] = -4.0                   # yb_dot
xinit[12] = -4.0                  # y2_dot

xterm = zeros(n)
xterm[1] = -lb/2           # xb
xterm[2] = sqrt(l1^2+l2^2) # yb
xterm[6] = -lb             # x2

init_mode = 1

# Reference Trajectory
Xref, Uref = reference_trajectory(model, times, t_trans, xinit, xterm, init_mode)

# Objective
Random.seed!(1)
Q = Diagonal([1.0;1.0;1.0; 1.0;1.0; 1.0;1.0; 1.0;1.0;1.0; 1.0;1.0; 1.0;1.0])
R = Diagonal(fill(1e-3,4))
Qf = Q

obj = map(1:N-1) do k
    LQRCost(Q,R,Xref[k],Uref[k])
end
push!(obj, LQRCost(Qf, R*0, Xref[N], Uref[1]))

# Define the NLP
nlp = HybridNLP(model, obj, init_mode, tf, N, Xref[1], Xref[end]);

## Solve

In [23]:
# Initial guess
Random.seed!(1)
Xguess = [x + 0.1*randn(length(x)) for x in Xref]
Uguess = [u + 0.1*randn(length(u)) for u in Uref]
Z0 = packZ(nlp, Xguess, Uguess);
# nlp = HybridNLP(model, obj, init_mode, tf, N, Xref[1], Xref[end], use_sparse_jacobian=false);

In [26]:
# Z_sol, solver = solve(Z0, nlp, c_tol=1e-6, tol=1e-6)
Z_sol, solver = solve(Z0, nlp, c_tol=1e-4, tol=1e-2)

Creating NLP Block Data...
Creating Ipopt...
Adding constraints...
starting Ipopt Solve...


This is Ipopt version 3.13.4, running with linear solver mumps.
NOTE: Other linear solvers might be more efficient (see Ipopt documentation).

Number of nonzeros in equality constraint Jacobian...:   248192
Number of nonzeros in inequality constraint Jacobian.:        0
Number of nonzeros in Lagrangian Hessian.............:        0

Total number of variables............................:      554
                     variables with only lower bounds:        0
                variables with lower and upper bounds:        0
                     variables with only upper bounds:        0
Total number of equality constraints.................:      552
Total number of inequality constraints...............:       62
        inequality constraints with only lower bounds:        0
   inequality constraints with lower and upper bounds:       62
        inequality constraints with only upper bounds:        0

iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
   0  

([-0.2500000000403383, 0.40355339059028017, 0.17453292519965125, 2.01377438093028e-11, 7.906379606575326e-12, -0.4999999999993803, 0.049999999998768724, -4.786701535577055e-12, -4.000000000001525, 9.38639184525435e-14  …  -6.89757525221992e-13, -0.4999999999717209, -6.931201083325183e-13, 1.7095293761140866e-12, -1.116177964217671e-12, 4.350430140123455e-14, -1.465554233291463e-12, 1.144743471340793e-13, -4.838870574448561e-12, 1.1491600260934224e-13], Ipopt.Optimizer)

In [27]:
Z_sol[1:14] - xinit

14-element Vector{Float64}:
 -4.0338288265218125e-11
 -2.993605363599272e-12
  2.182976022169214e-13
  2.01377438093028e-11
  7.906379606575326e-12
  6.197264923457624e-13
 -1.231278967672722e-12
 -4.786701535577055e-12
 -1.525002346625115e-12
  9.38639184525435e-14
  2.518573629420131e-12
  6.394884621840902e-13
  6.703631200362835e-14
 -2.0538518729654233e-14

In [28]:
Z_sol[end-13:end] - xterm

14-element Vector{Float64}:
 -1.786215619858922e-11
  8.664902129140728e-12
 -2.4579221580386657e-13
  9.16629713885005e-12
 -6.89757525221992e-13
  2.8279101282890906e-11
 -6.931201083325183e-13
  1.7095293761140866e-12
 -1.116177964217671e-12
  4.350430140123455e-14
 -1.465554233291463e-12
  1.144743471340793e-13
 -4.838870574448561e-12
  1.1491600260934224e-13

In [29]:
Z_sol

554-element Vector{Float64}:
 -0.2500000000403383
  0.40355339059028017
  0.17453292519965125
  2.01377438093028e-11
  7.906379606575326e-12
 -0.4999999999993803
  0.049999999998768724
 -4.786701535577055e-12
 -4.000000000001525
  9.38639184525435e-14
  ⋮
 -0.4999999999717209
 -6.931201083325183e-13
  1.7095293761140866e-12
 -1.116177964217671e-12
  4.350430140123455e-14
 -1.465554233291463e-12
  1.144743471340793e-13
 -4.838870574448561e-12
  1.1491600260934224e-13